# Can you find Extraterrestrial Intelligence (ETI)?
## A Breakthrough Listen Radio SETI Tutorial

In this tutorial, you'll be looking for a "signal" from an extraterrestrial transmitter in radio data using analysis software developed by the [Breakthrough Listen Initiative](https://breakthroughinitiatives.org/initiative/1) (BL). All of the data in this tutorial were simulated by the BL code [`setigen`](https://github.com/bbrzycki/setigen/blob/master/docs/source/index.rst) written by Bryan Brzycki; none of this is actual observational data, but it is based on the data formats and signal morphologies that we use in the actual BL project. 

There are dozens of interference (non-alien) signals that have been injected into this data, and a single "alien" signal - can you find ETI?

## `blimpy` and `turboSETI`

Before we do anything else, we need to download the BL-specific software that you'll be using for this project. 

### Downloading `blimpy`

The first package to download is called `blimpy`, which stands for "Breakthrough Listen I/O Methods for Python". The `blimpy` package helps us read, investigate, and plot `filterbank` (.fil) files, which you can think of as data cubes which store *power* values at each *frequency* we recorded over *time*. 

We are going to use the development version of blimpy hosted by the UC Berkeley SETI Center on [GitHub](https://github.com/UCBerkeleySETI/blimpy). Because we're using the dev version instead of the stable, `pip install`able version, we'll install this with git instead of pip. Go to a terminal window and use `cd` to navigate to your Week 7 folder (use `ls` to make sure you're in the right place). Now type `git clone https://github.com/UCBerkeleySETI/blimpy.git` (you can copy-paste the address from here, or the GitHub page using the `Code` button). You should see some messages about the code downloading, and, once it's finished, you should have a new directory called `blimpy` in your Week 7 folder. Use `cd` to navigate into it, and then type `pip install .` to install the code you just downloaded. This will take a minute or less and it should download (among itself, and other packages if necessary) something called `h5py`. `.h5` files are a more efficient way to store the same info as `.fil` files, and BL uses the external `h5py` software to do the writing/reading for this format.

### Downloading `turboSETI`

The second package we'll need to download is called `turboSETI` and is the official BL narrowband signal search pipeline. To break that down, turboSETI looks for very narrow frequency bands (~a few Hz) that contain anomalously high powers throughout an observation. It is even capable of following these signals if they drift off of their original frequency due to relative accelerations between the receiver on Earth and the transmitter out in space (very likely, as radial accelerations are *everywhere* in the universe). Finding these narrow signals would be exciting, because there's no astrophysical phenomenon that we know of that can get to bandwidths smaller than ~kHz (and those are radio masers, which are a very interesting topic in their own right). Not only that, but these very narrow signals are how we communicate to each other via GPS, cell signals, etc. on Earth! 

We are going to use my (Sofia Sheikh's) [personal version](https://github.com/Sofysicist/turbo_seti) of the `turboSETI` code, instead of the official version, because there are some features that I've developed, especially with the plotting in the end, that haven't been integrated into the official repository yet. To do this, we'll follow the same steps as we did for `blimpy`.

Go back to your terminal window and make sure you're in the Week 7 folder (use `ls` to make sure you're in the right place - you'll have to do a `cd ..` if you're still in the `blimpy` subfolder). Now type `git clone https://github.com/Sofysicist/turbo_seti.git` (again, you can copy-paste the address from here, or the GitHub page using the `Code` button). The code should download, and, once it's finished, you should have a new directory called `turbo_seti` in your Week 7 folder. Use `cd` to navigate into it, and then type `pip install .` to install the code. This install should be even quicker than `blimpy`.

Whew! Alright, now we're ready to import those packages, and jump into the analysis!

## Importing Packages / Other Preliminary Stuff 

Start with our standard command to get plots to show nicely in the notebook:

In [ ]:
%matplotlib inline

Now let's import those two BL packages we downloaded:

In [ ]:
import blimpy
import turbo_seti
import turbo_seti.find_doppler.seti_event as turbo
import turbo_seti.find_event as find

Time to tell the Jupyter notebook about our file names. You'll be searching the following six files for SETI signals:

In [ ]:
fil_filelist = ["on_guppi_58465_71709_on_0001.gpuspec.0000.fil",
               "off_guppi_58465_71836_off_0002.gpuspec.0000.fil",
               "on_guppi_58465_71963_on_0003.gpuspec.0000.fil",
               "off_guppi_58465_72089_off_0004.gpuspec.0000.fil",
               "on_guppi_58465_72216_on_0005.gpuspec.0000.fil", 
               "off_guppi_58465_72343_off_0006.gpuspec.0000.fil"]

Let's take a moment to understand what those filenames are telling us. 

- They are filterbank files, as described previously in the notebook (indicated by the `.fil` at the end), so they're storing frequency, power, and time information. 

- They were observed in an *on-off cadence*, meaning that, in the first file, we "observed" (remember, this is all *synthetic* or computer-generated data) the target-of-interest on the sky. Then, in the next file, we "moved the telescope" to an off-source position, or a random part of the sky far away from the primary target, and observed there for the same amount of time. We repeat that "on-off" pattern two more times. The idea is that any narrowband radio frequency interference (RFI) will appear no matter where the telescope is pointing, in both the on and the off observations, because it's coming from somewhere nearby on the surface of the Earth. BUT any actual signal generated by that position on the sky will only appear when we're pointed at the on-source. The filename tells you whether the observation was an "on" or an "off" source.

- "guppi" stands for the Green Bank Ultimate Pulsar Processing Instrument, which is a piece of hardware that helped to generate the files.

- After the "guppi", you'll see two strings of 5 numbers: this is the Modified Julian Date (MJD), which tells you the date and time when the observation was "started" (58465_71709 -> 58465.71709)

- The next number is the ID number of the observation - here, 0001 was the first observation of the on-source and 0006 was the last observation of the off-source

- "gpuspec" tells you the data was reduced with the GPU spectrometer

- The final four digits before the `.fil`, "0000", are a code for the BL data product with the highest frequency resolution and lowest time resolution (there's a trade-off here because of the uncertainty principle). Each frequency channel in this data product is about ~3 Hz wide, which is way narrower than what people would normally use for astronomy research. Remember, this is because we're looking for signals which we expect to be way narrower than what would be produced by nature! The trade-off is that each "time bin" is integrated over 18 seconds, so we lose out on the fine time-details that you would need to be able to observe ex. pulsars\*.

\*for studying pulsars, BL also saves a data product ending in "0002", which has high time resolution and "low" frequency resolution, and one intermediate-time intermediate-frequency product that ends in "0001" which is good for maser observations

## Performing the Initial Search for "Hits"

At its core, `turboSETI` is looking for frequency bins with anomalously high powers in each time bin of an observation, with some wiggle room to account for linear* drifts in frequency over time. But exactly how high of a power is "anomalous"? And how big of a drift do we want the code to allow? **You, as the scientist, need to make that decision!** But I can give you some details, and some guidelines.

**Maximum Drift Rate**: This parameter defines the largest "slopes", positive and negative, you want `turboSETI` to search in the data, and is measured in Hz/s. A signal with zero drift rate is likely RFI, and will appear as a vertical line in a "dynamic spectrum" or "waterfall plot" of the data. Signals with higher drift rates will appear as lines with greater and greater slopes. ~6 Hz/s used to be the standard value chosen for data at these frequencies (6 GHz or in the middle of so-called "C-band"). Breakthrough Listen tends to do ~4X this. But known M-dwarf exoplanetary systems could produce drifts of up to hundreds of Hz/s due to their extremely tight, high-acceleration orbits! What should you choose? I would recommend to start small and work your way up - with this algorithm, searching 2X as many drift rates will take 2X as long!

**Signal-to-Noise Ratio (SNR)**: This parameter is probably the one which you're a little more familiar with - the signal-to-noise ratio (SNR) gives a measure of the strength of a signal compared to the surrounding noise. Instrumental effects and statistical fluctuations produce a noise background that's mostly quiet, but has occasional strong spikes - we want to set our SNR high enough that we don't catch these false positives! The bigger problem, however, is RFI. The lower we set our SNR, the more random Earth-transmitters the algorithm will find. If you don't set your threshold high enough, you'll end up producing hundreds (or thousands! or millions!) of plots of RFI, and you won't be able to visually inspect them all for the needle-in-the-haystack SETI signal. BUT the higher you set your SNR, the more likely that you'll accidentally throw the real signal out - just because a signal is real doesn't mean it will be bright! The standard SNR choice in SETI searches is 10. BL has talked about using 5, which I'm doing for my current project. If you are finding that you're producing too many plots at the end of the notebook, a good way to fix that is to raise this value.

\*this is an approximation, but a good one when our observations are "short" (< a few hours)

**Go for it! Fill in the two parameters below with your best guesses!**

You can always clear out your output files (*if you don't do this, the output files won't write correctly!*) and try again if you want to play with the values later

In [ ]:
max_drift_rate = #Your number here! #Hz/s
signal_to_noise = #Your number here!

Alright, it's search time! We'll use the `FindDoppler` function to search all six files for hits using the parameters that you chose above. Each file will take ~a few seconds to a minute depending on the parameters you chose.

In [ ]:
for file in fil_filelist:
    print("working on " + file)
    find_seti_event = turbo.FindDoppler(file,
                                        max_drift=max_drift_rate, 
                                        snr=signal_to_noise)
    find_seti_event.search()

## Filtering the "Hits" into "Events"

After running that previous cell, you will notice that your Week 7 directory just gained a lot more files. There are three types of outputs that each `.fil` file produces when it's run through `FindDoppler`:

**.h5**: The files ending in `.h5` are conversions of the `.fil` files that are slightly more compressed than the original files. `FindDoppler` produces these because they're more efficient to search than the original `.fil` files.

**.log**: The files ending in `.log` are, well, log files! They provide verbose output of what occurred during the search. If you open one of them up in a text editor, you'll see a report of all of the drift rates that the algorithm searched, and which ones it detected "hits" in.

**.dat**: The `.dat` files are the ones that contain the results of the search. If you open one of them up in a text editor, you should see a header, which contains meta-data about the file you searched, followed by rows of data. Each row contains the parameters (signal-to-noise, drift rate, etc.) of a detected hit.

In a normal search, these .dat files could contain hundreds of thousands of hits, and almost every single one is going to be RFI. So we'll need to find some way to filter out the RFI and only show the most promising hits. Luckily, turboSETI has a tool for this: `find_events`!

In our jargon, an "event" is a series of related hits across observations. To group and filter our hits into events, we'll need to make some decisions about how to filter - this is where your judgement comes in again, on the following three parameters:

**check_zero_drift:** This parameter can be set to either True or False. If you set it to True, it will consider all hits, even those with zero drift. Remember that zero drift signals are extremely likely to be RFI, because they indicate no relative acceleration between the receiver and the transmitter. If you set it to False, it will not consider hits with zero drift.

**SNR_cut:** Here, you can either set the signal-to-noise ratio to the the value that you chose before (when you performed the `FindDoppler` search), OR you can choose a higher value. There are two reasons you might want to set a higher threshold: 1) You end up with too many hits to search by eye, and need to reduce the number of plots or, more importantly 2) You might want to require a stricter threshold for the on-sources while keeping a less strict threshold for the off-sources (which is what this essentially does). This technique allows you to catch direction-depending RFI which will still appear in the off-source, but weaker.

**filter_threshold:** This is really the key parameter in the `find_event` function. It has three valid values: 1, 2, and 3, corresponding to low, medium, and strict filtering, respectively. Here's a quick description of the three levels:

1. Returns hits above the SNR cut, taking into account the check_zero_drift parameter, but without an on-source/off-source check.

2. Returns hits that passed level 1 AND that are in at least *one* on-source but no off-sources. 

3. Returns events that passed level 2 AND that are present in *ALL* on-sources. 

**Go for it! Fill in the three parameters below with your best guesses!**

You can always clear out your output files (*if you don't do this, the output files won't write correctly!*) and try again if you want to play with the values later

In [ ]:
check_zero_drift = #Your value here!
filter_threshold = #Your value here!
SNR_cut = 12 #Your value here!

Now you can run the following cell as is - the first few lines define the .dat files we just made. Then we call the `find_events` function with the values that you defined above. Then we save the list of events as a `.csv` file.

In [ ]:
dat_file_list = ["on_guppi_58465_71709_on_0001.gpuspec.0000.dat", 
                 "off_guppi_58465_71836_off_0002.gpuspec.0000.dat",
                 "on_guppi_58465_71963_on_0003.gpuspec.0000.dat",
                 "off_guppi_58465_72089_off_0004.gpuspec.0000.dat",
                 "on_guppi_58465_72216_on_0005.gpuspec.0000.dat",
                 "off_guppi_58465_72343_off_0006.gpuspec.0000.dat"]

event_dataframe = find.find_events(dat_file_list, 
                       SNR_cut=SNR_cut, 
                       check_zero_drift=check_zero_drift, 
                       filter_threshold=filter_threshold, 
                       on_off_first='ON')

name = "synthetic"
id_num = "0"

if check_zero_drift == True:
    event_csv_string = name + '_' + id_num + '_f' + str(filter_threshold) + '_snr' + str(SNR_cut) + '_zero' + '.csv'
else:
    event_csv_string = name + '_' + id_num + '_f' + str(filter_threshold) + '_snr' + str(SNR_cut) + '.csv'

You should see multiple messages after the "All data loaded!" status line that tell you how many hits pass each step of the filtering process. If you end up with zero hits, you'll see a :(, and your dataframe will be empty - that means your filtering was too strict! You'll always want to examine your best candidates by eye, even if you have to loosen your restrictions to take a look at them. Re-try with less strict filtering.

If you instead see a message ending in "returning this table...", congratulations! You have some potential candidates! They're saved in a `pandas` dataframe, so let's check out their parameters below.

In [ ]:
event_dataframe

There's lots of information here, but most importantly, check out the Freq (frequency), SNR (signal-to-noise ratio), and DriftRate (drift rate) columns. Are there trends in those values?

Alright! Let's save this output as a csv using the `pandas` function `.to_csv`.

In [ ]:
event_dataframe.to_csv(event_csv_string)

## Plotting the data

It's the moment you've all been waiting for - let's plot our potential candidates and see if we've found any "signals" from extraterrestrial transmitters! We're going to use the `plot_event_pipeline` function to do this. This function cross-references the potential candidates saved in our `.csv` files with the original filterbank files that contain the full data-cube, in order to produce waterfall plots that show the full observation (on-source / off-source alternating). No alterations needed here, just run the cell below!

In [ ]:
fils_list_string = "fil_file_list.lst" #just a file containing the .fil names
turbo_seti.plot_event_pipeline.plot_event_pipeline(event_csv_string, 
                        fils_list_string,  
                        user_validation=False,
                        offset=0,
                        plot_snr=False)

Finally, take a look at either the plots created in the Jupyter notebook here, or the `.png` files that were saved in your Week 7 directory (they're the same). Here are a few tips for reading these waterfall plots:

- Time since the start of the observation is measured down the y-axis. The panels are in chronological order: on-off-on-off-on-off. 

- Frequency is shown along the x-axis. The 0.0 frequency in the center is the recorded "start frequency" for that event

- The color (z) axis shows the normalized power (scaled independently for each individual panel)

- The red dashed line is the "model" displaying the parameters of the recorded event. You'll notice that sometimes the model is offset from the start frequency, or has a different drift rate than the hit. Some of this is due to complexities about how the event gets recorded, and some of it is honestly due to small bugs from the code being a work-in-progress. Regardless, you should still be able to see the feature in the frame that the algorithm was matching the line to.

- RFI will look like a bright linear feature slicing through all six panels. An actual ETI signal would look like the same linear feature, but without appearing in the off panels.

Given all of that - do you think you found ETI? 
- If not, try again with different parameters. 
- If so... Congratulations! You found "aliens"! You have completed this tutorial :)